In [2]:
import pandas as pd
import numpy as np

# 1. Load your dataset
df = pd.read_csv("C:\\Users\\Chinmaya holla\\Desktop\\Airpollution\\Data\\Processed_data\\analysis_ready_data.csv")

# Ensure columns are numeric (handling any errors)
cols = ['SO2', 'NO2']
for col in cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# 2. Define CPCB Breakpoints for India (24-hour Average Standards)
# Format: (Low_Conc, High_Conc, Low_Index, High_Index)
# Source: CPCB National Air Quality Index
breakpoints_so2 = [
    (0, 40, 0, 50),
    (41, 80, 51, 100),
    (81, 380, 101, 200),
    (381, 800, 201, 300),
    (801, 1600, 301, 400),
    (1600, 10000, 401, 500)
]

breakpoints_no2 = [
    (0, 40, 0, 50),
    (41, 80, 51, 100),
    (81, 180, 101, 200),
    (181, 280, 201, 300),
    (281, 400, 301, 400),
    (400, 10000, 401, 500)
]

# 3. Define the Calculation Function
def calculate_sub_index(concentration, breakpoints):
    """
    Calculates the AQI sub-index based on concentration and breakpoints.
    Formula: I = I_lo + (I_hi - I_lo) * (C - C_lo) / (C_hi - C_lo)
    """
    if pd.isna(concentration):
        return np.nan
        
    for (Blo, Bhi, Ilo, Ihi) in breakpoints:
        if Blo <= concentration <= Bhi:
            return Ilo + (Ihi - Ilo) * (concentration - Blo) / (Bhi - Blo)
            
    # If it exceeds the highest defined range (rare)
    return 500 

# 4. Apply the calculation
df['SO2_AQI'] = df['SO2'].apply(lambda x: calculate_sub_index(x, breakpoints_so2))
df['NO2_AQI'] = df['NO2'].apply(lambda x: calculate_sub_index(x, breakpoints_no2))

# 5. Display the result
# Showing Date, Station, and the new Gas AQIs alongside your existing PM2.5 AQI
print(df[['Timestamp', 'Station', 'PM2_5_AQI', 'SO2_AQI', 'NO2_AQI']].head())

# Optional: Save to new CSV
output_path = "../data/Processed_data/Final_analysis_ready.csv"
df.to_csv(output_path, index=False)

    Timestamp      Station  PM2_5_AQI  SO2_AQI     NO2_AQI
0  01-01-2019  BapujiNagar      157.0  10.6250  500.000000
1  02-01-2019  BapujiNagar      142.0  10.0000   96.557436
2  03-01-2019  BapujiNagar      147.0  11.3375   99.698462
3  04-01-2019  BapujiNagar      152.0  10.7750  102.450000
4  05-01-2019  BapujiNagar      138.0  10.2375  102.870000
